In [1]:
import skimage 
from skimage import exposure, measure, feature, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
img = cv2.imread('Intelligent Systems in Medicine/Project/ISIC2018_Task3_Training_Input/ISIC_0024306.jpg')
img2 = cv2.imread('Intelligent Systems in Medicine/Project/ISIC2018_Task3_Training_Input/ISIC_0024307.jpg')

In [4]:
df = pd.read_csv('Intelligent Systems in Medicine/Project/HAM10000_metadata.csv')

In [5]:
df = df.sort_values(by=['image_id'])
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
4349,HAM_0000550,ISIC_0024306,nv,follow_up,45.0,male,trunk
4263,HAM_0003577,ISIC_0024307,nv,follow_up,50.0,male,lower extremity
4217,HAM_0001477,ISIC_0024308,nv,follow_up,55.0,female,trunk
3587,HAM_0000484,ISIC_0024309,nv,follow_up,40.0,male,trunk
1451,HAM_0003350,ISIC_0024310,mel,histo,60.0,male,chest


In [6]:
df['contrast'] = 'default value'
df['dissimilarity'] = 'default value'
df['homogeneity'] = 'default value'
df['energy'] = 'default value'
df['correlation'] = 'default value'

df['average_red'] = 'default value'
df['average_green'] = 'default value'
df['average_blue'] = 'default value'


df['perimeter'] = 'default value'
df['symmetry'] = 'default value'
df['area_variance1'] = 'default value'
df['area_variance2'] = 'default value'
df['area_variance3'] = 'default value'

df['area'] = 'default value'
#df['m10'] = 'default value'
#df['m01'] = 'default value'
#df['m20'] = 'default value'
#df['m11'] = 'default value'
#df['m02'] = 'default value'
#df['m30'] = 'default value'
#df['m21'] = 'default value'
#df['m12'] = 'default value'
#df['m03'] = 'default value'
#df['mu20'] = 'default value'
#df['mu11'] = 'default value'
#df['mu02'] = 'default value'
#df['mu03'] = 'default value'
#df['mu21'] = 'default value'
##df['mu12'] = 'default value'
#df['mu03'] = 'default value'
#df['nu20'] = 'default value'
#df['nu30'] = 'default value'
#df['nu21'] = 'default value'
#df['nu12'] = 'default value'
#df['nu03'] = 'default value'



In [7]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,contrast,dissimilarity,homogeneity,...,correlation,average_red,average_green,average_blue,perimeter,symmetry,area_variance1,area_variance2,area_variance3,area
4349,HAM_0000550,ISIC_0024306,nv,follow_up,45.0,male,trunk,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
4263,HAM_0003577,ISIC_0024307,nv,follow_up,50.0,male,lower extremity,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
4217,HAM_0001477,ISIC_0024308,nv,follow_up,55.0,female,trunk,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
3587,HAM_0000484,ISIC_0024309,nv,follow_up,40.0,male,trunk,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
1451,HAM_0003350,ISIC_0024310,mel,histo,60.0,male,chest,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value


In [33]:
df.loc[df['image_id'] == 'ISIC_00' + '24306', ['average_red']] = 0.7220961237576853

In [35]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,contrast,dissimilarity,homogeneity,...,correlation,average_red,average_green,average_blue,perimeter,symmetry,area_variance1,area_variance2,area_variance3,area
4349,HAM_0000550,ISIC_0024306,nv,follow_up,45.0,male,trunk,12,default value,default value,...,default value,0.722096,default value,default value,default value,default value,default value,default value,default value,default value
4263,HAM_0003577,ISIC_0024307,nv,follow_up,50.0,male,lower extremity,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
4217,HAM_0001477,ISIC_0024308,nv,follow_up,55.0,female,trunk,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
3587,HAM_0000484,ISIC_0024309,nv,follow_up,40.0,male,trunk,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
1451,HAM_0003350,ISIC_0024310,mel,histo,60.0,male,chest,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value


In [41]:
#24306-34320 (-34321)
for image_id in range(24306,34321):
    
    img = cv2.imread('Intelligent Systems in Medicine/Project/ISIC2018_Task3_Training_Input/ISIC_00' + str(image_id) +'.jpg')
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (17, 17), 32)
    ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    im2, contours,hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key=cv2.contourArea)
    if len(cnt) > 4:
        ellipse = cv2.fitEllipse(cnt)
        x,y,w,h = cv2.boundingRect(cnt)
        area = hsv[int(y+(0.3*h)):int(y+(0.8*h)),int((0.2*w)+x):int((0.7*w)+x)]
        ellipse_cnt = cv2.ellipse2Poly( (int(ellipse[0][0]),int(ellipse[0][1]) ) ,( int(ellipse[1][0]),int(ellipse[1][1]) ),int(ellipse[2]),0,360,1)
       
    
    # add average color information to dataframe
    red_sum = sum(sum(np.multiply(img[:,:,0],thresh)))
    green_sum = sum(sum(np.multiply(img[:,:,0],thresh)))
    blue_sum = sum(sum(np.multiply(img[:,:,0],thresh)))

    area_sum = sum(sum(thresh))
    
    
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['average_red']] = red_sum/area_sum
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['average_green']] = green_sum/area_sum
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['average_blue']] = blue_sum/area_sum
    
    #add moment information to dataframe
    
    moments = cv2.moments(cnt, True)
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['area']] = moments['m00']

    # add perimeter, symmetry and area variance
    perimeter = cv2.arcLength(cnt,True)
    comp = cv2.matchShapes(cnt,ellipse_cnt,1,0.0)
    variance = cv2.meanStdDev(area)
    
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['perimeter']] = perimeter
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['symmetry']] = comp
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['area_variance1']] = variance[1][0][0]
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['area_variance2']] = variance[1][1][0]
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['area_variance3']] = variance[1][2][0]

    
    # add texture information to dataframe
    gCoMat = feature.greycomatrix(gray,[2],[0], 256, symmetric = True, normed = True)

    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['contrast']] = feature.greycoprops(gCoMat, prop='contrast')
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['dissimilarity']] = feature.greycoprops(gCoMat, prop='dissimilarity')
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['homogeneity']] = feature.greycoprops(gCoMat, prop='homogeneity')
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['energy']] = feature.greycoprops(gCoMat, prop='energy')
    df.loc[df['image_id'] == 'ISIC_00' + str(image_id), ['correlation']] = feature.greycoprops(gCoMat, prop='correlation')
    
    
    

In [40]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,contrast,dissimilarity,homogeneity,...,correlation,average_red,average_green,average_blue,perimeter,symmetry,area_variance1,area_variance2,area_variance3,area
4349,HAM_0000550,ISIC_0024306,nv,follow_up,45.0,male,trunk,23.2521,2.99571,0.314132,...,0.987946,0.793936,0.793936,0.793936,1175.28,0.0562844,85.3257,14.1578,29.3723,55753
4263,HAM_0003577,ISIC_0024307,nv,follow_up,50.0,male,lower extremity,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
4217,HAM_0001477,ISIC_0024308,nv,follow_up,55.0,female,trunk,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
3587,HAM_0000484,ISIC_0024309,nv,follow_up,40.0,male,trunk,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
1451,HAM_0003350,ISIC_0024310,mel,histo,60.0,male,chest,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value


In [42]:
df.to_csv('features.csv')